In [1]:
# only going to upload single files,
# since the single large file will be too large
# without splitting it up

# (using python 3.8 in this notebook instead of 3.10 for snowpark)


In [2]:
import configparser
from snowflake.snowpark import Session

In [3]:
config = configparser.ConfigParser()
config.read('../../snowpark.config')

connection_parameters = {
    "account": config['snowflake']['account'],
    "user": config['snowflake']['user'],
    "password": config['snowflake']['password'],
    "role": config['snowflake']['role'],
    "warehouse": config['snowflake']['warehouse'],
    "database": config['snowflake']['database'],
    "schema": config['snowflake']['schema']
}

session = Session.builder.configs(connection_parameters).create()

In [9]:
# Create a temp stage.
_ = session.sql("create or replace stage xml_tests.xml_loading.xml_stage").collect()

In [4]:
put_result = session.file.put("../xml_datasets/xml_micro/*.xml", "@xml_tests.xml_loading.xml_stage/small_xml_files")
put_result[0].status

'UPLOADED'

In [6]:
session.sql("""create table xml_tests.xml_loading.xml_landing 
(xml_src variant);""").collect()

[Row(status='Table XML_LANDING successfully created.')]

In [2]:

# copy into from stage
# _ = session.sql("""
# COPY INTO xml_tests.xml_loading.xml_landing
#      FROM @xml_tests.xml_loading.xml_stage/small_xml_files
#     file_format = (TYPE =  XML  STRIP_OUTER_ELEMENT = TRUE)
# """).collect()

In [ ]:
#example of query to flatten xml in table:
# _ = session.sql(
# """SELECT 
# xml_flat.value:"$" as element_value
# ,xml_flat.value:"@" as element_name
# ,xml_flat.value:"@date" as performance_score_date
# FROM xml_landing,
# LATERAL FLATTEN( INPUT => xml_src:"$" ) xml_flat;"""
# )

# timed in snowflake, just to look at execution time
# flattening time: 1.9s
